# empyrical-integration

This example shows how to integrate pinkfish with the empyrical library.

In [1]:
import datetime

import empyrical as em
import matplotlib.pyplot as plt
import pandas as pd
from talib.abstract import *

import pinkfish as pf

# Format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = '^GSPC'
capital = 10000
#start = datetime.datetime(1900, 1, 1)
start = datetime.datetime(*pf.SP500_BEGIN)
end = datetime.datetime.now()

Define Strategy Class - sell in may and go away

In [4]:
class Strategy:

    def __init__(self, symbol, capital, start, end):
        self.symbol = symbol
        self.capital = capital
        self.start = start
        self.end = end
        
        self.ts = None
        self.rlog = None
        self.tlog = None
        self.dbal = None
        self.stats = None

    def _algo(self):
        pf.TradeLog.cash = capital

        for i, row in enumerate(self.ts.itertuples()):

            date = row.Index.to_pydatetime()
            high = row.high; low = row.low; close = row.close; 
            end_flag = pf.is_last_row(self.ts, i)

            # Buy (at the open on first trading day in Nov).
            if self.tlog.shares == 0:
                if row.month == 11 and row.first_dotm:
                    self.tlog.buy(date, row.open)
            # Sell (at the open on first trading day in May).
            else:
                if ((row.month == 5 and row.first_dotm) or end_flag):
                    self.tlog.sell(date, row.open)

            # Record daily balance
            self.dbal.append(date, high, low, close)

    def run(self):
        
        # Fetch and select timeseries.
        self.ts = pf.fetch_timeseries(self.symbol)
        self.ts = pf.select_tradeperiod(self.ts, self.start, self.end,
                                        use_adj=True)
        # Add calendar columns.
        self.ts = pf.calendar(self.ts)
        
        # Finalize timeseries.
        self.ts, self.start = pf.finalize_timeseries(self.ts, self.start)
        
        # Create tlog and dbal objects
        self.tlog = pf.TradeLog(symbol)
        self.dbal = pf.DailyBal()
        
        # Run algorithm, get logs
        self._algo()
        self._get_logs()
        self._get_stats()

    def _get_logs(self):
        self.rlog = self.tlog.get_log_raw()
        self.tlog = self.tlog.get_log()
        self.dbal = self.dbal.get_log(self.tlog)

    def _get_stats(self):
        s.stats = pf.stats(self.ts, self.tlog, self.dbal, self.capital)

Run Strategy

In [5]:
s = Strategy(symbol, capital, start, end)
s.run()

Run Benchmark, Retrieve benchmark logs, and Generate benchmark stats

In [6]:
benchmark = pf.Benchmark(symbol, s.capital, s.start, s.end)
benchmark.run()

Empyrical API

In [7]:
# Filter warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Convert pinkfish data to Empyrical format
returns = s.dbal['close'].pct_change()
returns.index = returns.index.tz_localize('UTC')

benchmark_rets = benchmark.dbal['close'].pct_change()
benchmark_rets.index = benchmark_rets.index.tz_localize('UTC')

# Get metrics
aggregate_returns = em.aggregate_returns(returns, 'yearly')
alpha = em.alpha(returns, benchmark_rets)
alpha_aligned = em.alpha_aligned(returns, benchmark_rets)
alpha_beta = em.alpha_beta(returns, benchmark_rets)
alpha_beta_aligned = em.alpha_beta_aligned(returns, benchmark_rets)
annual_return = em.annual_return(returns)
annual_volatility = em.annual_volatility(returns)
beta = em.beta(returns, benchmark_rets)
beta_aligned = em.beta_aligned(returns, benchmark_rets)
cagr = em.cagr(returns)
calmar_ratio = em.calmar_ratio(returns)
capture = em.capture(returns, benchmark_rets)
conditional_value_at_risk = em.conditional_value_at_risk(returns)
cum_returns = em.cum_returns(returns)
cum_returns_final = em.cum_returns_final(returns)
down_alpha_beta = em.down_alpha_beta(returns, benchmark_rets)
down_capture = em.down_capture(returns, benchmark_rets)
downside_risk = em.downside_risk(returns)
excess_sharpe = em.excess_sharpe(returns, benchmark_rets)
max_drawdown = em.max_drawdown(returns)
omega_ratio = em.omega_ratio(returns)
sharpe_ratio = em.sharpe_ratio(returns)
simple_returns = em.simple_returns(s.dbal['close'])
sortino_ratio = em.sortino_ratio(returns)
stability_of_timeseries = em.stability_of_timeseries(returns)
tail_ratio = em.tail_ratio(returns)
up_alpha_beta = em.up_alpha_beta(returns, benchmark_rets)
up_capture = em.up_capture(returns, benchmark_rets)
up_down_capture = em.up_down_capture(returns, benchmark_rets)
value_at_risk = em.value_at_risk(returns)

In [8]:
# Store results in a pf.Series
s = pd.Series(dtype=object)
s['aggregate_returns'] = aggregate_returns
s['alpha'] = alpha
s['alpha_aligned'] = alpha_aligned
s['alpha_beta'] = alpha_beta
s['alpha_beta_aligned'] = alpha_beta_aligned
s['annual_return'] = annual_return
s['annual_volatility'] = annual_volatility
s['beta'] = beta
s['beta_aligned'] = beta_aligned
s['cagr'] = cagr
s['calmar_ratio'] = calmar_ratio
s['capture'] = capture
s['conditional_value_at_risk'] = conditional_value_at_risk
s['cum_returns'] = cum_returns
s['cum_returns_final'] = cum_returns_final
s['down_alpha_beta'] = down_alpha_beta
s['down_capture'] = down_capture
s['downside_risk'] = downside_risk
s['excess_sharpe'] = excess_sharpe
s['max_drawdown'] = max_drawdown
s['omega_ratio'] = omega_ratio
s['sharpe_ratio'] = sharpe_ratio
s['simple_returns'] = simple_returns
s['sortino_ratio'] = sortino_ratio
s['stability_of_timeseries'] = stability_of_timeseries
s['tail_ratio'] = tail_ratio
s['up_alpha_beta'] = up_alpha_beta
s['up_capture'] = up_capture
s['up_down_capture'] = up_down_capture
s['value_at_risk'] = value_at_risk

In [9]:
# Display metrics
df = pd.DataFrame(s, columns=['value'])
df

,value
aggregate_returns,1957 -0.01 1958 0.17 1959 0.09 1960 ...
alpha,0.03
alpha_aligned,0.03
alpha_beta,"[0.029197390380382604, 0.5043664492118572]"
alpha_beta_aligned,"[0.029197390380382604, 0.5043664492118572]"
annual_return,0.08
annual_volatility,0.13
beta,0.50
beta_aligned,0.50
cagr,0.08
